In [1]:
pip install tensorflow


  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4847 sha256=2fe5bfc4d72d46330b4bf5507bec980906ba954872f539c08ab8f67e18a258b4
  Stored in directory: c:\users\melys\appdata\local\pip\cache\wheels\a0\16\9c\5473df82468f958445479c59e784896fa24f4a5fc024b0f501
Successfully built termcolor


In [22]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.7.0


In [23]:
#charger les donnees
#Convertir les exemples de données d'entiers en nombres à virgule flottante
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [24]:
#modèle par empilement de couches
# layuers liste de couches a ajouter au modele comme CONV2D et LSTM
#classe de modele de base
# fct d'activation par defaut lineaire a(x)=a 
# flatten applatir l'entree (batch,) -> (batch,1) ajoute une dimension supplémentaire   #data form
#couche entière connectée à l'intérieur permet de construire tout un tas de couches cachees
# dropout couche dabondon  
#définit de manière aléatoire les unités d'entrée sur 0 avec une fréquence de rate à chaque étape pendant le temps d'entraînement
#ce qui permet d'éviter le surapprentissage.
#Les entrées non définies sur 0 sont augmentées de 1/(1 - taux) de sorte que la somme de toutes les entrées reste inchangée.
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)), 
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])


In [25]:
#modèle retourne un vecteur de logits ou log-odds scores, un pour chaque classe.

predictions = model(x_train[:1]).numpy()
predictions

array([[-0.3040493 , -0.3307922 , -0.15652788, -0.2333492 ,  0.437234  ,
         0.13650489, -0.7426105 , -0.12658675,  0.57200855, -1.0313811 ]],
      dtype=float32)

In [26]:
#fonction convertit ces logits à des probabilités pour chaque classe:
#Softmax convertit un vecteur de valeurs en une distribution de probabilité.
#Les éléments du vecteur de sortie sont dans la plage (0, 1) et la somme à 1.
#Chaque vecteur est géré indépendamment. L' axisargument définit sur quel axe de l'entrée la fonction est appliquée.
#Softmax est souvent utilisé comme activation pour la dernière couche d'un réseau de classification 
#car le résultat pourrait être interprété comme une distribution de probabilité.
#le softmax de chaque vecteur x est calculé comme exp(x) / tf.reduce_sum(exp(x)).
tf.nn.softmax(predictions).numpy()

array([[0.07947999, 0.07738264, 0.09211399, 0.08530264, 0.16679886,
        0.12347757, 0.05126172, 0.09491369, 0.1908644 , 0.03840446]],
      dtype=float32)

In [27]:
#Définir une fonction de perte pour la formation à l' aide losses.SparseCategoricalCrossentropy , qui prend un vecteur de logits 
#et un True index et retourne une perte scalaire pour chaque exemple.
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
#Cette perte est égale au log de probabilité négatif de la vraie classe : 
#La perte est nulle si le modèle est sûr de la bonne classe.
#Ce modèle donne sans formation probabilités proche aléatoire (1/10 pour chaque classe), 
#de sorte que la perte initiale devrait être proche de -tf.math.log(1/10) ~= 2.3 .

In [28]:
loss_fn(y_train[:1], predictions).numpy()

2.0916958

In [29]:
# optimiseur adam algo de descente de gradient stochastique
#« efficace du point de vue informatique, nécessite peu de mémoire, est invariante au redimensionnement diagonal des gradients 
#et convient bien aux problèmes volumineux en termes de données/paramètres ».
# la perte loss_fn
# mesure a evaluer accuracy (la précision)
#liste des métriques à évaluer par le modèle lors de l'entraînement et des tests.
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [30]:
#Entraîne le modèle pour un nombre fixe d'époques (itérations sur un jeu de données).
#méthode pour ajuster vos paramètres du modèle et de minimiser la perte
# epochs nombre d'iterations
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2951 - accuracy: 0.9141
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1415 - accuracy: 0.9575
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1074 - accuracy: 0.9673
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0852 - accuracy: 0.9743
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0733 - accuracy: 0.9765


In [31]:
#La Model.evaluate méthode vérifie les performances des modèles, généralement sur une « validation-set » ou « Test-set ».
# Renvoie la valeur de perte et les valeurs des métriques pour le modèle en mode test.
#verbose le mode 
#verbose = 1, qui comprend à la fois une barre de progression et une ligne par époque (par defaut)
#verbose = 0, signifie silencieux
#verbose = 2, une ligne par époque, c'est-à-dire n° d'époque/n° total. des époques
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0752 - accuracy: 0.9771 - 360ms/epoch - 1ms/step


[0.07517688721418381, 0.9771000146865845]

In [32]:
#Si vous souhaitez que votre modèle renvoie une probabilité, 
#vous pouvez envelopper le modèle entraîné et lui attacher le softmax 
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [21]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[4.87319767e-08, 1.59964531e-09, 1.44165256e-06, 3.23446511e-05,
        1.61473161e-11, 1.35747470e-07, 1.79582575e-12, 9.99964595e-01,
        8.69174471e-07, 6.13895509e-07],
       [3.04531568e-07, 3.78727736e-06, 9.99990225e-01, 4.68011694e-06,
        2.03307067e-14, 7.27946201e-07, 6.19277287e-08, 5.00613060e-13,
        1.07242393e-07, 2.58545355e-12],
       [1.91150320e-07, 9.99255121e-01, 1.98959780e-04, 1.37356237e-05,
        5.46996489e-05, 7.24996141e-07, 1.31544641e-06, 3.87280743e-04,
        8.15875464e-05, 6.48276273e-06],
       [9.99762118e-01, 5.32308020e-09, 1.87216774e-05, 8.77046944e-07,
        5.13422310e-06, 2.97671450e-05, 1.65786041e-04, 1.55591115e-05,
        1.11419496e-07, 1.97243207e-06],
       [1.36755170e-06, 2.51397525e-09, 4.41341435e-05, 1.62646749e-08,
        9.98291075e-01, 2.50765364e-09, 3.93432356e-06, 1.01441699e-04,
        2.20761626e-06, 1.55583106e-03]], dtype=float32)>